In [1]:

# -*- coding: utf-8 -*-
# ============================================================================
# Hollow-fiber membrane 1D model (Mass balance and Pressure drop)
# Modelo 1D de membrana de fibra oca
#
# Chu et al paper can be found here: >>>>>> https://doi.org/10.1016/j.cherd.2019.05.054 <<<<<<
# ----------------------------------------------------------------------------
# EN: This script solves a simple 1D axial mass-transfer model for a hollow-fiber
#     membrane taking into account a simple pressure drop due to flow
#     inside a tube. The lumen and shell channels exchange
#     species via a membrane with area per unit length AREA_PER_L.
#     The ODEs for the species in the shell [du_i/dz , dP/dz]
#     and bore [dv_i/dz, dp/dz] are integrated using scipy's solve_bvp.
#
# PT: Este script resolve um modelo simples de transferência de massa axial
#     unidimensional para uma membrana de fibra oca
#     levando em consideração uma queda de pressão simples devido ao fluxo
#     em um tubo. Os canais do lúmen e da parede trocam
#     espécies através de uma membrana com área por unidade de comprimento AREA_PER_L.
#     As EDOs para as espécies na parede [du_i/dz , dP/dz]
#     e no interior [dv_i/dz, dp/dz] são integradas usando a função solve_bvp do scipy.
# ----------------------------------------------------------------------------
# Considerations:
# ·Herning & Zipperer mixing rule works best for gases of hidrocarbons,
#   not suited when there's significant portion of hydrogen
# ·Not considering cross-flow at the end of the membrane that does not use sweep gas Cl.
# ·Isothermal, considering the Joule-Thompson effect negligible
# ·Pressure drop calculations assumes square pitch
# ·Assuming plug flow in permeate and feed side (very reasonable at least for bore side)
# ----------------------------------------------------------------------------
# Note:
# I believe divergences in results are partially due to the calculation of the mean viscosity
#   and partially due to the author's using orthogonal colocation and RK4
# ============================================================================


import numpy as np
from scipy.integrate import solve_bvp

#todo: Generalize size for [ M, MU, Q, comp_f, comp_s]
# get mass and viscosities from hysys

# --- Physical constants ---
R = 8.314      # J/(mol·K)
M = np.array([44.01e-3, 16.04e-3,28.02e-3]) # Molar Mass [CO2, CH4,N2] (kg/mol)
MU = np.array([1.48e-5, 1.11e-5,2.85e-5])  # Viscosities [CO2, CH4,N2] (Pa·s)
SQRT_M = np.sqrt(M)
# --------------------------

# --- Problem Data --------
T = 298                                         # Temperature (K)
P_f = 5e5                                       # Feed Pressure (Pa)
p_p = p_s = 1e5                                 # Permeate Pressure (Pa) / Sweep Pressure (Pa)
Q = np.array([8.405e-9, 1.323e-10,3.968e-10])   # Permeances [CO2, CH4,N2] (mol/(m^2·Pa·s)
f_total = 4.464e-4                              # Feed flow (mol/s)
comp_f = np.array([0.4, 0.6,0.0])               # Feed molar fraction
s_flow = 2.012e-5                               # Sweep gas flow (mol/s)
comp_s = np.array([0.0, 0.0,1.0])               # Sweep molar fraction
# --------------------------

f_in = f_total * comp_f # List of flow per species [CO2, CH4,N2] in (mol/s)
n_comp = len(comp_f) # Number of species

# --- Project design variables ---
Current = "Counter"             # 'Counter' or 'Co'
Feed = "Bore"                   # 'Bore' or 'Shell'
L = 0.3                         # Fiber Length (m)
N = 106                         # Number of fibers
D_o = 200e-6                    # Fiber outer diameter (m)
D_i = 150e-6                    # Fiber Inner diameter (m)
D = 0.012                       # Module internal diameter (m)
# --------------------------

AREA_PER_L = np.pi * D_o * N # Fibers total cylinder outer shell area per unit length


def get_mean_mix_viscosity(fractions):
    """
    Viscosity of mixture (Herning & Zipperer) along whole membrane
    https://en.wikipedia.org/wiki/Viscosity_models_for_mixtures#Classic_mixing_rules
    fractions has shape (n_comp, n_nodes)
    """
    numerator = np.sum(fractions * MU[:, np.newaxis] * SQRT_M[:, np.newaxis], axis=0)
    denominator = np.sum(fractions * SQRT_M[:, np.newaxis], axis=0)
    # safeguard for zero division
    return numerator / np.maximum(denominator, 1e-12)



def model_odes(z, y):
    """
    Defines our ODEs system for mass and momentum balance
    number of variables: 2*n_comp + 2 (species on both sides plus 2 pressures)
    """

    Q_col = Q[:, np.newaxis]
    u = y[0:n_comp] # (n_comp, n_nodes)
    v = y[n_comp:-2] # (n_comp, n_nodes)
    P = y[-2]   # (n_nodes,) - Shell Pressure
    p = y[-1]   # (n_nodes,) - Bore Pressure

    u_total = np.sum(u, axis=0)
    v_total = np.sum(v, axis=0)

    # --- Safeguards for numerical stability ---
    u_total_safe = np.maximum(u_total, 1e-12)
    v_total_safe = np.maximum(v_total, 1e-12)

    x = u / u_total_safe
    y_frac = v / v_total_safe

    partial_pressure_shell = P * x
    partial_pressure_tube = p * y_frac
    driving_force = partial_pressure_shell - partial_pressure_tube
    flux_per_length = AREA_PER_L * Q_col * driving_force

    # --- Momentum balance (pressure drop) ---
    mu_v_mix = get_mean_mix_viscosity(y_frac)
    mu_u_mix = get_mean_mix_viscosity(x)

    # 192 is for shell, 128 is for fibers.
    # Pressure need to decay along the flow, so, for flows in the opposite direction of z, its positive
    dP_dz = ( 192 * N * D_o * (D + N * D_o) * mu_u_mix * R * T * u_total_safe /
             (np.pi * (D ** 2 - N * D_o ** 2) ** 3 * P) )
    dp_dz = -(128 * R * T * mu_v_mix * v_total_safe) / (np.pi * D_i ** 4 * N * p)

    if Current == "Counter":
      du_dz = +flux_per_length
      dv_dz = +flux_per_length
    else:
      du_dz = -flux_per_length
      dv_dz = +flux_per_length
      dP_dz = -dP_dz
    return np.vstack((du_dz, dv_dz, dP_dz, dp_dz))

def model_bc_sweep(y_a, y_b):
    """
    Definition of boundary conditions residue
    Considering sweep.
    y_a: Conditions at z = 0
    y_b: Conditions at z = L
    """

    if Feed == "Shell" and Current == "Counter":
      if s_flow == 0:
        bc_a = bc1 = y_a[n_comp:-2] - s_flow * comp_s       # v(0) = s_in
        bc_b = bc2 = y_b[0:n_comp] - f_in                   # u(L) = f_in
        bc_b_pressure_P = bc3 = np.array([ y_b[-2] - P_f ]) #P(L) = P_feed
        bc_b_pressure_p = bc4 = np.array([ y_b[-1] - p_p ]) #p(L) = P_perm
      else:
        bc_a = bc1 = y_a[n_comp:-2] - s_flow * comp_s       # v(0) = s_in
        bc_b = bc2 = y_b[0:n_comp] - f_in                   # u(L) = f_in
        bc_b_pressure_P = bc3 = np.array([ y_b[-2] - P_f ]) #P(L) = P_feed
        bc_a_pressure_p = bc4 = np.array([ y_a[-1] - p_s ]) #p(0) = P_sweep

    elif Feed == "Bore" and Current == "Counter":
      if s_flow == 0:
        bc_a = bc1 = y_a[n_comp:-2] - f_in                  # v(0) = f_in
        bc_b = bc2 = y_b[0:n_comp] - s_flow * comp_s        # u(L) = s_in
        bc_a_pressure_P = bc3 = np.array([ y_a[-2] - p_p ]) #P(0) = P_perm
        bc_a_pressure_a = bc4 = np.array([ y_a[-1] - P_f ]) #p(0) = P_feed
      else:
        bc_a = bc1 = y_a[n_comp:-2] - f_in                  # v(0) = f_in
        bc_b = bc2 = y_b[0:n_comp] - s_flow * comp_s        # u(L) = s_in
        bc_b_pressure_P = bc3 = np.array([ y_b[-2] - p_s ]) #P(L) = P_sweep
        bc_a_pressure_p = bc4 = np.array([ y_a[-1] - P_f ]) #p(0) = P_feed

    elif Feed == "Bore" and Current == "Co":
      if s_flow == 0:
        bc_a = bc1 = y_a[n_comp:-2] - f_in                  # v(0) = f_in
        bc_a2 = bc2 = y_a[0:n_comp] - s_flow * comp_s       # u(0) = s_in
        bc_b_pressure_P = bc3 = np.array([ y_b[-2] - p_p ]) #P(L) = P_perm
        bc_a_pressure_p = bc4 = np.array([ y_a[-1] - P_f ]) #p(0) = P_feed
      else:
        bc_a = bc1 = y_a[n_comp:-2] - f_in                  # v(0) = f_in
        bc_a2 = bc2 = y_a[0:n_comp] - s_flow * comp_s       # u(0) = s_in
        bc_a_pressure_P = bc3 = np.array([ y_a[-2] - p_s ]) #P(0) = P_sweep
        bc_a_pressure_p = bc4 = np.array([ y_a[-1] - P_f ]) #p(0) = P_feed

    elif Feed == "Shell" and Current == "Co":
      if s_flow == 0:
        bc_a = bc1 = y_a[n_comp:-2] - s_flow * comp_s       # v(0) = s_in
        bc_a2 = bc2 = y_a[0:n_comp] - f_in                  # u(0) = f_in
        bc_a_pressure_P = bc3 = np.array([ y_a[-2] - P_f ]) #P(0) = P_feed
        bc_b_pressure_p = bc4 = np.array([ y_b[-1] - p_p ]) #p(L) = P_perm
      else:
        bc_a = bc1 = y_a[n_comp:-2] - s_flow * comp_s       # v(0) = s_in
        bc_a2 = bc2 = y_a[0:n_comp] - f_in                  # u(0) = f_in
        bc_a_pressure_P = bc3 = np.array([ y_a[-2] - P_f ]) #P(0) = P_feed
        bc_a_pressure_p = bc4 = np.array([ y_a[-1] - p_s ]) #p(0) = P_sweep

    return np.concatenate((bc1, bc2,bc3,bc4))


# --- 2. Setup solver arguments ---
z_nodes = np.linspace(0, L, 100) # initial nodes

y_guess = np.zeros((2*n_comp + 2, z_nodes.size)) # initial guess of size (n_var,nodes.size)
for i in range(n_comp):
  y_guess[i, :] = f_in[i]
  y_guess[n_comp + i, :] = (s_flow * comp_s)[i]
y_guess[-2, :] = [(P_f+p_p)/2]*z_nodes.size # Shell P
y_guess[-1, :] = [(P_f+p_p)/2]*z_nodes.size # Bore p

# --- 3. Solve using BVP ---
print(f"Solving BVP for Scenario {5}")
sol = solve_bvp(model_odes, model_bc_sweep, z_nodes, y_guess, # BVP takes the derivatives and bc residues functions
                verbose=0, tol = 1e-4, bc_tol=1e-4, max_nodes=1000)
#adjusts node size for the desired precision to be met, without exceeding max_nodes (1k is the default)

def print_results(sol,bool_print = True):
  if sol.success and bool_print:
      print("Solution found.\n")

      # --- Extract Results ---

      # Results for (z=0) u,v,p,P,x,y and totals
      u_0 = sol.y[0:n_comp, 0] # u(0)
      u_0_total = np.sum(u_0)
      x_0 = u_0 / u_0_total # x(0)

      v_0 = sol.y[n_comp:-2, 0]  # v(0)
      v_0_total = np.sum(v_0)
      y_0 = v_0 / v_0_total # y(0)

      P_0 = sol.y[-2, 0] # P(0)
      p_0 = sol.y[-1, 0]  #p(0)

      # Results for (z=L) u,v,p,P,x,y and totals
      u_L = sol.y[0:n_comp, -1]  # u(L)
      u_L_total = np.sum(u_L)
      x_L = u_L / u_L_total  # x(L)

      v_L = sol.y[n_comp:-2, -1]  # v(L)
      v_L_total = np.sum(v_L)
      y_L = v_L / v_L_total  # y(L)

      P_L = sol.y[-2, -1]  # P(L)
      p_L = sol.y[-1, -1]  # p(L)

      # Chu Paper's Table 5 Results
      paper_r_total = 4.209e-4
      paper_r_ch4 = 63.45
      paper_p_total = 4.567e-5
      paper_p_co2 = 55.37

      print("--- Results comparison ---")
      print(f"\nCurrent: {Current} | Feed: {Feed}")
      print("\nRESULTS FOR z = 0")
      print(f"| Parameter             |     Shell     |     Bore      | ")
      print(f"|-----------------------|---------------|---------------|")
      print(f"| Total flow (mol/s)    | {u_0_total:<13.4e} | {v_0_total:<13.4e} |")
      print(f"| Composition CO2 (mol%)| {x_0[0]*100:<13.2f} | {y_0[0]*100:<13.2f} |")
      print(f"| Composition CH4 (mol%)| {x_0[1]*100:<13.2f} | {y_0[1]*100:<13.2f} |")
      print(f"| Composition N2  (mol%)| {x_0[2]*100:<13.2f} | {y_0[2]*100:<13.2f} |")
      print(f"| Pressure (bar)        | {P_0/1e5:<13.3f} | {p_0/1e5:<13.3f} |")

      print("\nRESULTS FOR z = L")
      print(f"| Parameter             |      Shell    |      Bore     | ")
      print(f"|-----------------------|---------------|---------------|")
      print(f"| Total flow (mol/s)    | {u_L_total:<13.4e} | {v_L_total:<13.4e} |")
      print(f"| Composition CO2 (mol%)| {x_L[0]*100:<13.2f} | {y_L[0]*100:<13.2f} |")
      print(f"| Composition CH4 (mol%)| {x_L[1]*100:<13.2f} | {y_L[1]*100:<13.2f} |")
      print(f"| Composition N2  (mol%)| {x_L[2]*100:<13.2f} | {y_L[2]*100:<13.2f} |")
      print(f"| Pressure (bar)        | {P_L/1e5:<13.3f} | {p_L/1e5:<13.3f} |")

      print("\nRESULTS FOR CHU's PAPER")
      print(f"| Parameter             |      Retentate     |      Permeate     | ")
      print(f"|-----------------------|--------------------|-------------------|")
      print(f"| Total flow (mol/s)    | {paper_r_total:<13.4e}      |  {paper_p_total:<13.4e}    |")
      print(f"| Composition CO2 (mol%)| (N/A)              | {paper_p_co2:<17} |")
      print(f"| Composition CH4 (mol%)| {paper_r_ch4:<17}  | (N/A)             |")

      print("\n--- PRESSURE DROP ---")
      print(f"SHELL SIDE:")
      print(f"  P_0 (z=0): {P_0/1e5:.3f} bar")
      print(f"  P_L (z=L): {P_L/1e5:.3f} bar")
      print(f"  DROP (dP):      {abs(P_L - P_0)/100:.3f} mbar")

      print(f"\nBORE SIDE:")
      print(f"  p_0 (z=0): {p_0/1e5:.3f} bar")
      print(f"  p_L (z=L): {p_L/1e5:.3f} bar")
      print(f"  DROP (dp):      {abs(p_L - p_0)/100:.3f} mbar")

      print("\n--- MASS BALANCE VERIFICATION: ---")
      total_in = f_total + np.sum(s_flow * comp_s)

      if Current == "Counter":
          total_out = v_L_total + u_0_total
      else:
          total_out = v_L_total + u_L_total

      print(f"Total in  (mol/s): {total_in:.5f}")
      print(f"Total out (mol/s):   {total_out:.5f}")
      print(f"Balance error:        {total_in - total_out:.2e}")

  else:
      print("BVP Solver failed to find a solution.")
      print(f"Message: {sol.message}")

print_results(sol)

Solving BVP for Scenario 5
Solution found.

--- Results comparison ---

Current: Counter | Feed: Bore

RESULTS FOR z = 0
| Parameter             |     Shell     |     Bore      | 
|-----------------------|---------------|---------------|
| Total flow (mol/s)    | 4.6249e-05    | 4.4640e-04    |
| Composition CO2 (mol%)| 55.82         | 40.00         |
| Composition CH4 (mol%)| 1.74          | 60.00         |
| Composition N2  (mol%)| 42.44         | -0.00         |
| Pressure (bar)        | 1.000         | 5.000         |

RESULTS FOR z = L
| Parameter             |      Shell    |      Bore     | 
|-----------------------|---------------|---------------|
| Total flow (mol/s)    | 2.0120e-05    | 4.2027e-04    |
| Composition CO2 (mol%)| 0.00          | 36.34         |
| Composition CH4 (mol%)| 0.00          | 63.54         |
| Composition N2  (mol%)| 100.00        | 0.12          |
| Pressure (bar)        | 1.000         | 4.936         |

RESULTS FOR CHU's PAPER
| Parameter          